In [8]:
from copy import *
from random import *
from jogos import *
from utils import *
import string

from collections import namedtuple

stateBreakthru = namedtuple('EstadoBreakthru', 'to_move')

class EstadoBreakthru(stateBreakthru):
    
    def verificar_terminou(self):
        # 1 = White,  2 = Black
        if self.to_move == 2:
            for row in range(len(self.board)):
                if (self.board[0][row] == 'W'):
                    return 1
        elif self.to_move == 1:
            for row in range(len(self.board)):
                if (self.board[7][row] == 'B'):
                    return 1    
        return 0
    
    def pos_convert(self, i, j):
        alfabeto = "abcdefgh"
        return alfabeto[j] + str(8 - i)

    def move_convert(self, move):
        numbers = []
        alfabeto = "abcdefgh"
        numbers.append(int(alfabeto.rfind(move[0])))
        numbers.append(int(move[1]) - 1)
        return numbers

    def __init__(self, to_move=1, n=8):
        self.board = [['.' for i in range(n)] for i in range(n)]
        #self.terminou = self.verificar_terminou()
        for i in range(2):
            for j in range(8):
                self.board[i][j] = 'B'

        for i in range(6, 8):
            for j in range(8):
                self.board[i][j] = 'W'


    def moves(self, to_move):
        list_moves = set()
        for i in range(0,8):
            for j in range(0,8):
                if self.board[i][j] == 'B' and to_move == 2 and j+1 <= 7:
                    if self.board[i+1][j] == '.' and i+1 <= 7:
                       list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j)))
                    if (self.board[i+1][j+1] == 'W' or self.board[i+1][j+1] == '.') and i+1 <= 7 and j+1 <= 7:
                       list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j+1)))
                    if (self.board[i+1][j-1] == 'W' or self.board[i+1][j-1] == '.') and i+1 <=7 and j-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j-1)))
                if self.board[i][j] == 'B' and to_move == 2 and j+1 == 8:
                    if self.board[i+1][j] == '.' and i+1 <= 7:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j)))
                    if (self.board[i+1][j-1] == 'W' or self.board[i+1][j-1] == '.') and i+1 <= 7:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j-1)))           

                if self.board[i][j] == 'W' and to_move == 1 and j+1 <= 7:
                    if self.board[i-1][j] == '.' and i-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i-1,j)))
                    if (self.board[i-1][j+1] == 'B' or self.board[i-1][j+1] == '.') and i-1 >= 0 and j+1 <= 7:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i-1,j+1)))
                    if (self.board[i-1][j-1] == 'B' or self.board[i-1][j-1] == '.') and i-1 >= 0 and j-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i-1,j-1)))
                if self.board[i][j] == 'W' and to_move == 1 and j+1 == 8:
                    if self.board[i-1][j] == '.' and i-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i-1,j)))
                    if (self.board[i-1][j-1] == 'B' or self.board[i-1][j-1] == '.') and i-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i-1,j-1)))
        return sorted(list_moves)

## CLASS JOGO ##
class JogoBT(Game):
    """Play Breakthrough on an 8 x 8 board, with Max (first player) playing 'W' (1).
    """
    def __init__(self):
        self.initial = EstadoBreakthru(to_move=1)
        self.board = [['.' for i in range(8)] for i in range(8)]
        for i in range(2):
            for j in range(8):
                self.board[i][j] = 'B'

        for i in range(6, 8):
            for j in range(8):
                self.board[i][j] = 'W'

    def actions(self, state):
        "Legal moves for B and W"
        #Apenas queremos ver as ações possíveis para o jogador atual (to_move)
        return state.moves(state.to_move)

    def result(self, state, move):
        if state.to_move == 1:
            index_start = state.move_convert(move[0:2])
            new_index = state.move_convert(move[3:5]) #talvez esteja errado
            state.board[7 - index_start[1]][index_start[0]] = '.'
            state.board[7 - new_index[1]][new_index[0]] = 'W'
            novo_estado = EstadoBreakthru(to_move=2, n=8)
            novo_estado.board = state.board.copy()
            return novo_estado
        else:
            index_start = state.move_convert(move[0:2])
            new_index = state.move_convert(move[3:5]) #talvez esteja errado
            state.board[7 - index_start[1]][index_start[0]] = '.'
            state.board[7 - new_index[1]][new_index[0]] = 'B'
            novo_estado = EstadoBreakthru(to_move=1, n=8)
            novo_estado.board = state.board.copy()
            return novo_estado

    def terminal_test(self, state):
        "A state is terminal if the whites reached line 8 or some black reached line 1."
        return state.verificar_terminou == 1

    def display(self, state):
        symbols = "abcdefgh"
        print("-"*17)
        for i in range(len(state.board)):
            print("{num}|".format(num=8-i) + ' '.join(state.board[i]))
        print("-+" + "-"*15)
        print(" |" + ' '.join(i for i in symbols))

        if state.verificar_terminou == 1:
            print("True")
            return
        elif state.to_move == 1:
            print("--NEXT PLAYER: W")
        elif state.to_move == 2:
            print("--NEXT PLAYER: B")

    def executa(self, estado, listaJogadas):
        "executa varias jogadas sobre um estado dado"
        "devolve o estado final "
        s = estado
        for j in listaJogadas:
            s = self.result(s, j)
        return s


In [9]:
accoes = "a2-a3 a7-a6 c2-c3 a6-b5 b2-b3 b5-b4".split()
jj = JogoBT()
interessante=jj.executa(jj.initial, accoes)
print("O que está lá?")
print(interessante.board[1][0])
novas_accoes = jj.actions(interessante)
outrointeressante=jj.result(interessante,novas_accoes[0])
print("O que está lá?")
print(interessante.board[1][0])
jj.display(interessante)
print("Jogadas possíveis: ", novas_accoes)

AttributeError: 'EstadoBreakthru' object has no attribute 'copy'